In [1]:
import networkx as nx
import os

# Import given number of files
# NUMBER_OF_GRAPHS_WANTED=-1, to import all files

def import_some(NUMBER_OF_GRAPHS_WANTED,path):
    # Change the directory
    os.chdir(path)
  
    # Read text File

    l = []
  
    def read_file(file_path):
        l.append(nx.read_graphml(file_path))
  
  
    # iterate through all file
    i = -1
    for file in os.listdir():
        i+=1
        if i == NUMBER_OF_GRAPHS_WANTED:
            break
        # Check whether file is in text format or not
        if file.endswith(".graphml"):
            file_path = f"{path}\{file}"
 
            # call read text file function
            read_file(file_path)
    return l


In [2]:
def node_data_table(graph):
    df = pd.DataFrame.from_dict(dict(graph.nodes(data=True)), orient='index')
    df.drop(columns=['dn_position_x', 'dn_position_y', 'dn_position_z'], inplace=True)
    return df

In [3]:
def node1_name(node_df, row):
    print(row)
    print("....")
    node_id = row["node1"]
    name = node_df[node_df["dn_correspondence_id"] == node_id][dn_name]
    print(name)
    print(type(name))
    return name

In [4]:
import pandas as pd

def edge_dataframe(number, path):
    
    graph_list = import_some(number, path)
    edges = []
    fiber_length = []
    FA = []
    number_of_fibers = []
    node1 = []
    node2 = []
    
    for graph in graph_list:
        for n1, n2, data in graph.edges.data():
            edges.append((n1,n2))
            node1.append(int(n1))
            node2.append(int(n2))
            fiber_length.append(data['fiber_length_mean'])
            FA.append(data['FA_mean'])
            number_of_fibers.append(data['number_of_fibers'])
            
    
            
    data = {"edges":edges, "node1":node1, "node2":node2, "fiber_length": fiber_length, "FA": FA, "number_of_fibers":number_of_fibers}
    df = pd.DataFrame(data=data)
    
    edge_confidence_series = df["edges"].value_counts()
    edge_confidence_series = edge_confidence_series.to_frame(name="edge confidence")
    
    df_mean = df.groupby("edges").mean()
    df_median = df.groupby("edges").median()
    
    inner_joined_total = df_mean.join(
    df_median,
    lsuffix="_mean",
    rsuffix="_median")
    
    inner_joined_total = inner_joined_total.merge(edge_confidence_series, right_index=True, left_index=True)
    
    os.chdir('..')
    os.mkdir("tables_new")
    
    inner_joined_total.rename(columns = {"node1_mean":"node1", "node2_mean":"node2"}, inplace = True)
    
    inner_joined_total.drop(columns=['node1_median', 'node2_median'], inplace=True)
    
    
    
    
    node_df = node_data_table(graph_list[0])
    node_dict = {}
    
    for index, row in node_df.iterrows():
        node_dict[str(row["dn_correspondence_id"])] = row["dn_name"]

    
    
    
    
    inner_joined_total["name_node1"] = inner_joined_total.apply(lambda x:node_dict[str(int(x["node1"]))],axis=1)
    inner_joined_total["name_node2"] = inner_joined_total.apply(lambda x:node_dict[str(int(x["node2"]))],axis=1)
    
    
    
    inner_joined_total.to_csv("tables_new\\t0.csv", index=True, sep=';',float_format='%.3f')
    
    
#     node_df.to_csv("node_data_table.csv", index=True, sep=';',float_format='%.3f')
    
    
    
    return inner_joined_total

In [7]:
def table_100(main_path, graph_path):
    df = edge_dataframe(-1, main_path+graph_path)
    path_table = main_path + "\\tables_new\\"
    path_freq = main_path+"\\frequency_graphs\\"
    for i in range(1,101):
        file_path = path_freq + "g{}.graphml".format(i)
        g  = nx.read_graphml(file_path)
        df_i = df[df.index.isin(g.edges())]
        df_i.to_csv(path_table+"t{}.csv".format(i), index=True, sep=';',float_format='%.3f')

In [9]:
main_path = "..."
graph_path = "..."
table_100(main_path, graph_path)